In [3]:
from datetime import datetime

import pandas as pd

import xgboost as xgb

import numpy as np

import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.feature_selection import SelectPercentile, mutual_info_classif

In [4]:
# Read CSV
df = pd.read_csv('../data/DontGetKicked/training.csv')

# Show df info
df.info()

# Show feature statistics
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72983 entries, 0 to 72982
Data columns (total 34 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   RefId                              72983 non-null  int64  
 1   IsBadBuy                           72983 non-null  int64  
 2   PurchDate                          72983 non-null  object 
 3   Auction                            72983 non-null  object 
 4   VehYear                            72983 non-null  int64  
 5   VehicleAge                         72983 non-null  int64  
 6   Make                               72983 non-null  object 
 7   Model                              72983 non-null  object 
 8   Trim                               70623 non-null  object 
 9   SubModel                           72975 non-null  object 
 10  Color                              72975 non-null  object 
 11  Transmission                       72974 non-null  obj

,RefId,IsBadBuy,VehYear,VehicleAge,WheelTypeID,VehOdo,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,BYRNO,VNZIP1,VehBCost,IsOnlineSale,WarrantyCost
count,72983.000000,72983.000000,72983.000000,72983.000000,69814.000000,72983.000000,72965.000000,72965.000000,72965.000000,72965.000000,72668.000000,72668.000000,72668.000000,72668.000000,72983.000000,72983.000000,72983.000000,72983.000000,72983.000000
mean,36511.428497,0.122988,2005.343052,4.176644,1.494299,71499.995917,6128.909217,7373.636031,8497.034332,9850.928240,6132.081287,7390.681827,8775.723331,10145.385314,26345.842155,58043.059945,6730.934326,0.025280,1276.580985
std,21077.241302,0.328425,1.731252,1.712210,0.521290,14578.913128,2461.992768,2722.491986,3156.285284,3385.789541,2434.567723,2686.248852,3090.702941,3310.254351,25717.351219,26151.640415,1767.846435,0.156975,598.846788
min,1.000000,0.000000,2001.000000,0.000000,0.000000,4825.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,835.000000,2764.000000,1.000000,0.000000,462.000000
25%,18257.500000,0.000000,2004.000000,3.000000,1.000000,61837.000000,4273.000000,5406.000000,6280.000000,7493.000000,4275.000000,5414.000000,6536.000000,7784.000000,17212.000000,32124.000000,5435.000000,0.000000,837.000000
50%,36514.000000,0.000000,2005.000000,4.000000,1.000000,73361.000000,6097.000000,7303.000000,8444.000000,9789.000000,6062.000000,7313.000000,8729.000000,10103.000000,19662.000000,73108.000000,6700.000000,0.000000,1155.000000
75%,54764.500000,0.000000,2007.000000,5.000000,2.000000,82436.000000,7765.000000,9021.000000,10651.000000,12088.000000,7736.000000,9013.000000,10911.000000,12309.000000,22808.000000,80022.000000,7900.000000,0.000000,1623.000000
max,73014.000000,1.000000,2010.000000,9.000000,3.000000,115717.000000,35722.000000,36859.000000,39080.000000,41482.000000,35722.000000,36859.000000,39080.000000,41062.000000,99761.000000,99224.000000,45469.000000,1.000000,7498.000000


In [5]:
categorical_columns = [
    'Auction', 'Make', 'Model', 'Trim', 'SubModel', 'Color', 
    'VehYear', 'Transmission', 'WheelType', 'Nationality', 'Size', 
    'TopThreeAmericanName', 'PRIMEUNIT', 'AUCGUART', 'VNST',
    'WheelTypeID', 'VNZIP1', 'IsOnlineSale'
]

df[categorical_columns] = df[categorical_columns].astype('category')

In [11]:
for column in categorical_columns:
    print(df[column].value_counts())
    print()

Auction
MANHEIM    41043
OTHER      17501
ADESA      14439
Name: count, dtype: int64

Make
CHEVROLET       17248
DODGE           12912
FORD            11305
CHRYSLER         8844
PONTIAC          4258
KIA              2484
SATURN           2163
NISSAN           2085
HYUNDAI          1811
JEEP             1644
SUZUKI           1328
TOYOTA           1144
MITSUBISHI       1030
MAZDA             979
MERCURY           913
BUICK             720
GMC               649
HONDA             497
OLDSMOBILE        243
ISUZU             134
VOLKSWAGEN        134
SCION             129
LINCOLN            97
INFINITI           42
VOLVO              37
CADILLAC           33
ACURA              33
LEXUS              31
SUBARU             28
MINI               24
PLYMOUTH            2
HUMMER              1
TOYOTA SCION        1
Name: count, dtype: int64

Model
PT CRUISER              2329
IMPALA                  1990
TAURUS                  1425
CALIBER                 1375
CARAVAN GRAND FWD V6    1289
     

In [24]:
for column in categorical_columns:
    print((df[column].value_counts() / len(df[column])))
    print()

Auction
MANHEIM    0.562364
OTHER      0.239796
ADESA      0.197841
Name: count, dtype: float64

Make
CHEVROLET       0.236329
DODGE           0.176918
FORD            0.154899
CHRYSLER        0.121179
PONTIAC         0.058342
KIA             0.034035
SATURN          0.029637
NISSAN          0.028568
HYUNDAI         0.024814
JEEP            0.022526
SUZUKI          0.018196
TOYOTA          0.015675
MITSUBISHI      0.014113
MAZDA           0.013414
MERCURY         0.012510
BUICK           0.009865
GMC             0.008892
HONDA           0.006810
OLDSMOBILE      0.003330
ISUZU           0.001836
VOLKSWAGEN      0.001836
SCION           0.001768
LINCOLN         0.001329
INFINITI        0.000575
VOLVO           0.000507
CADILLAC        0.000452
ACURA           0.000452
LEXUS           0.000425
SUBARU          0.000384
MINI            0.000329
PLYMOUTH        0.000027
HUMMER          0.000014
TOYOTA SCION    0.000014
Name: count, dtype: float64

Model
PT CRUISER              0.031912
IMPAL